In [127]:
import numpy as np
import scipy.linalg as la
import time
import bezout_5 as bz

#TEX_DIR = '/home/jp/Documents/Bezout/bezout/tex/txt'
TEX_DIR = '../tex/txt'

deg = [4, 4]
with open(TEX_DIR+'/deg.txt', 'w') as f:
    f.write(str(deg))
    
t = 25
m = 16000
n = len(deg)

Field = QQ
R = PolynomialRing(Field, 'x', n)
x = R.gens()
xx = [x[0]**0] + list(x)

fshape = [d+1 for d in deg]
dx = [(i+1)*deg[i] for i in range(n)]
dy = [(n-i)*deg[i] for i in range(n)]
fn, Dx, Dy = factorial(n), prod(dx), prod(dy)
with open(TEX_DIR+'/Dx.txt', 'w') as f:
    f.write("{0:d}".format(Dx))


P = [bz.rand_poly(n-1, m, deg, t, x) for i in range(n)] + xx
#P = load('P.sobj')

In [128]:
save(P, 'P')
bz.P2txt(n, deg, P, TEX_DIR)
F = [bz.poly2prism(fshape, p) for p in P]

In [129]:
t = time.clock()
Gx, Gy, Hx, Hy = bz._GH(n, fn, deg, dx, dy)
H, K = bz._HK(n, Hx, Hy)
J = bz._J(Dx, Dy, F, n, fshape, dx, dy, Gx, Gy)
C = bz._C(n, Dx, Dy, J)
B = bz._B(n, Dx, Dy, H, K, C)
B = bz.block_triang(n, Dx, Dy, dx, dy, deg, B)
construction_B_time = time.clock() - t
with open(TEX_DIR+'/construction_B_time.txt', 'w') as f:
    f.write("{0:.4f}".format(construction_B_time))

In [130]:
bb = B.reshape(n*Dx+Dx, Dy)

In [131]:
np.savetxt('bb.txt', bb, fmt='%d')

In [86]:
BB = []
for k in range(n+1):
        Bk = matrix(Field, B[k])
        BB.append(Bk[:, :])
bz.bz2txt(n, TEX_DIR, BB)

In [87]:
print(BB[0].LLL().str())

[  0   0   0   0   0   0   0   0]
[  0   0   0   0   0   0   0   0]
[  0   0  23  38  -1   0  20   0]
[-23 -38  26 -18   7   0  12   0]
[  0   0 -23 -38   1   0  42   0]
[ 23  38  36  -6  -7   0 -22   0]
[  0   0   0   0 -62   0   6   0]
[ 62 -24 -23  16   1   0 -12   0]


In [14]:
np.savetxt('b0.txt', u, fmt='%d')

In [33]:
"""
computing rank of B0
"""
t = time.clock()
r0 = rank(BB[0])
rank_B0_time = time.clock() - t

with open(TEX_DIR+'/rank_B0.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/rank_B0_time.txt', 'w') as f:
    f.write("{0:.4f}".format(rank_B0_time))
print("sage_rank = {0:d}".format(r0))

bb = []
for k in range(n+1):
    bb.append(np.array(BB[k], dtype=float))

b0 = bb[0]
numpy_rank = np.linalg.matrix_rank(b0)
print("numpy_rank = {0:d}".format(numpy_rank))

sage_rank = 659
numpy_rank = 659


In [34]:
"""
reduction of Bezoutian matrices
"""
epsi = abs(b0).max()/1.0e5
#epsi = 1.0/1.0e8

t = time.clock()
nb_relations = 1
while nb_relations > 0:
    bb, r0, nb_relations = bz.iteration(bb, r0, epsi)
    print bb[0].shape, r0, nb_relations
print "bbt"
bbt = []
for k in range(n+1):
    bbt.append(bb[k].T)
#bbt = normaliz(bbt, epsi)
nb_relations = 1
while nb_relations > 0:
    bbt, r0, nb_relations = bz.iteration(bbt, r0, epsi)
    print bbt[0].shape, r0, nb_relations
reductions_time = time.clock() - t
with open(TEX_DIR+'/bezout_dim.txt', 'w') as f:
    f.write("{0:d}".format(r0))
with open(TEX_DIR+'/reductions_time.txt', 'w') as f:
    f.write("{0:.4f}".format(reductions_time))

(720, 702) 654 18
(720, 696) 654 6
(720, 696) 654 0
bbt
(696, 720) 654 0


In [ ]:
"""
Numerical compation of the roots
"""
t = time.clock()
bbt_roots = bz.bbt2roots(bbt, epsi, r0)
compute_roots_time = time.clock() - t
with open(TEX_DIR+'/compute_roots_time.txt', 'w') as f:
    f.write("{0:.4f}".format(compute_roots_time))

In [35]:
test_bbtr = bz.roots_test(P, x, bbt_roots)
test_roots = np.sort(test_bbtr[:])

np.savetxt(TEX_DIR+'/test_roots.txt', test_roots,  fmt='%1.3e')

hist, bin_edges = np.histogram(np.log10(test_roots), bins='scott')


with open(TEX_DIR+'/histogram.txt', 'w') as f:
    for k in range(len(hist)):
        nb_roots = hist[k]
        left_bin, right_bin = bin_edges[k], bin_edges[k+1]
        if nb_roots > 0:
            if k < len(hist)-1:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\\\\\n".format(left_bin, right_bin, nb_roots))
            else:
                f.write("$[{0:2.1f}, {1:2.1f}]$ & ${2:d}$\n".format(left_bin, right_bin, nb_roots))

In [36]:
hist, bin_edges

(array([ 12,  44, 123, 135, 117,  75,  62,  31,  18,  17,   7,   5,   0,
          0,   3,   1,   0,   3,   0,   0,   0,   0,   0,   0,   0,   1]),
 array([-14.43527217, -14.17594673, -13.91662128, -13.65729584,
        -13.3979704 , -13.13864496, -12.87931951, -12.61999407,
        -12.36066863, -12.10134319, -11.84201774, -11.5826923 ,
        -11.32336686, -11.06404142, -10.80471597, -10.54539053,
        -10.28606509, -10.02673965,  -9.7674142 ,  -9.50808876,
         -9.24876332,  -8.98943788,  -8.73011243,  -8.47078699,
         -8.21146155,  -7.95213611,  -7.69281066]))

In [28]:
t = time.clock()
GB, grobner_dim = bz.compute_grobner(R, P, n)
grobner_time = time.clock() - t
bz.gb2txt(TEX_DIR, GB)
with open(TEX_DIR+'/grobner_dim.txt', 'w') as f:
    f.write("{0}".format(grobner_dim))
with open(TEX_DIR+'/grobner_time.txt', 'w') as f:
    f.write("{0:.4f}".format(grobner_time))
print(grobner_dim)

+Infinity


In [9]:
with open(TEX_DIR+'/bezout_exact.txt', 'w') as f:
    X = matrix(ZZ, Dx, 0)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    Y, Y_time = bz.BB2Y(BB, X_ortho)
    f.write("nb de Y-relations = {0:d}\n".format(Y.nrows()))

    
    K = bz.Y2K(BB[0], X_ortho, Y)
    X_dim = Dx - K.nrows()
    f.write("X_dim = {0:d}\n".format(X_dim))

    
    Y_ortho = Y.right_kernel_matrix().LLL()
    X, X_time = bz.BB2X(BB, Y_ortho)
    N = bz.X2N(BB[0], Y_ortho, X)
    X_ortho = X.kernel().basis_matrix().LLL().transpose()
    f.write("nb de X-relations = {0:d}\n".format(X.ncols()))

    
    XBBY = []
    for k in range(n+1):
        XBBY.append(X_ortho.transpose()*BB[k]*Y_ortho.transpose())
    bezout_exact_dim = rank(XBBY[0])
    f.write("bezout_exact_dim = {0:d}\n".format(bezout_exact_dim))

    
    K_ortho = XBBY[0].kernel().basis_matrix().right_kernel_matrix()
    N_ortho = N.kernel().basis_matrix().transpose()

    KBBN = []
    for k in range(n+1):
        KBBN.append(K_ortho*XBBY[k]*N_ortho)

    XX = []
    for k in range(n):
        xx = KBBN[0].solve_right(KBBN[k+1])
        XX.append(xx)

    test_XX = [norm(bz.X2p(XX, Field, p)) for p in P[:n]]
    
    f.write("test_XX = {0:s}".format(test_XX))


In [10]:
test_XX

[0, 0, 0]

In [37]:
bz.pol2hom4ps2(n, P, "../HOM4PS2")

In [91]:
bz.pol2bertini(n, P, "../bertini")

In [42]:
M = Matrix([[1, -1], [3, 4], [0, 2]])
type(M)

<type 'sage.matrix.matrix_integer_dense.Matrix_integer_dense'>

In [45]:
BB[0].rank()

659